# Generate a Corpus of British Literature, 1840-1860

Here I'll be using the database I generated for the Corpus-DB project to generate a corpus of British literature from around 1840-1860, roughly surrounding our 1851 census data. 

In [2]:
import sqlite3

In [4]:
dbPath = '/home/jon/Code/gitenberg-scrape/pg-text-7.db'

In [5]:
conn = sqlite3.connect(dbPath)

In [6]:
cur = conn.cursor()

Grab all texts with the Library of Congress category (LCC) of "PR" (British literature) that GoodReads tells us were published between 1840 and 1860. Limit to only those written in English.

In [120]:
cur.execute("""SELECT title, id, gr_pubDate 
               from meta 
               where "LCC" like "%PR%" 
               and gr_pubDate > 1839 and gr_pubdate < 1860 
               and languages like "%en%" """)

In [36]:
data = cur.fetchall()

In [38]:
len(data)

132

In [50]:
sorted(data)

[('A Book of Nonsense', '13646.0', '1846--'),
 ("A Child's Dream of a Star", '42232.0', '1850-3-18'),
 ('A Christmas Carol', '19337.0', '1843--'),
 ('A Christmas Carol', '24022.0', '1843--'),
 ('A Christmas Carol in Prose; Being a Ghost Story of Christmas',
  '46.0',
  '1843-12-19'),
 ('A Christmas Carol: The original manuscript', '30368.0', '1843--'),
 ('A Fair Penitent', '2006.0', '1857-7-18'),
 ('A House to Let', '2324.0', '1858--'),
 ('A Tale of Two Cities', '98.0', '1859--'),
 ('Adam Bede', '507.0', '1859--'),
 ("Agatha's Husband: A Novel", '21767.0', '1853--'),
 ('Agnes Grey', '767.0', '1847--'),
 ('Alton Locke, Tailor and Poet: An Autobiography', '8374.0', '1850--'),
 ('Amours De Voyage', '1393.0', '1858--'),
 ('Ancient Poems, Ballads, and Songs of the Peasantry of England',
  '649.0',
  '1846--'),
 ('Antonina; Or, The Fall of Rome', '3606.0', '1850--'),
 ("Arthur O'Leary: His Wanderings And Ponderings In Many Lands",
  '32424.0',
  '1845--'),
 ('Autobiographic Sketches', '7306.

In [98]:
# Remove duplicates.
# 1. If there is more than one book with the same title, take the book with the highest ID.
# (I.e., the one released by PG the latest, since it's more likely to be a better edition.)
# 2. If a book has a "complete" edition, and also exists in several volumes, throw out
# the volumes in favor of the complete edition. 
dupes = [19337, 46, 30368, 2432, 6054, 20795, 678, 12954, 22496, 4401, 4402, 4404]

What if we look for works with publication dates learned from Wikipedia? 

In [42]:
cur.execute('SELECT title, id, wp_publication_date from meta where "LCC" like "%PR%" and languages like "%en%"')

In [43]:
wpData = cur.fetchall()

In [48]:
list(filter((lambda x: x[2] != ''), wpData))

[("Alice's Adventures in Wonderland", '11.0', '1865-11-26'),
 ('Peter Pan', '16.0', '--10-11'),
 ('The Voyage Out', '144.0', '1915-03-26'),
 ('Black Beauty', '271.0', '1877-11-24'),
 ('Prince Otto, a Romance', '372.0', '1885-11-01'),
 ('The Warden', '619.0', '1855-01-05'),
 ("The Haunted Man and the Ghost's Bargain", '644.0', '1848-12-19'),
 ('The Cricket on the Hearth: A Fairy Tale of Home', '678.0', '1845-12-20'),
 ("Alice's Adventures in Wonderland: HTML Edition", '928.0', '1865-11-26'),
 ('Night and Day', '1245.0', '1919-10-20'),
 ('Jane Eyre: An Autobiography', '1260.0', '1847-10-16'),
 ('Pride and Prejudice', '1342.0', '1813-01-28'),
 ('The Talisman', '1377.0', '1825-06-22'),
 ('The Black Dwarf', '1460.0', '1816-12-02'),
 ('Kenilworth', '1606.0', '1821-01-08'),
 ('The Adventures of Sherlock Holmes', '1661.0', '1892-10-14'),
 ('Zuleika Dobson; Or, An Oxford Love Story', '1845.0', '1911-10-26'),
 ('The Red House Mystery', '1872.0', '1922-04-06'),
 ('Piccadilly Jim', '2005.0', '1917

This doesn't really help--there are only a few works between 1840 and 1860 here, and I think I already have them in the other collection.

See if books in Corpus-DB have Wikipedia categories like "1840 British Novels." 

In [56]:
wpSubjectPatterns = ["%s_novels" % year for year in range(1840, 1860)] + \
  ["%s_British_novels" % year for year in range(1840, 1860)]

In [67]:
queryFrags = ['or wp_subjects like "%' + subj + '%"' for subj in wpSubjectPatterns]

In [73]:
queryFrags[0] = queryFrags[0][3:] # Chop off the "or" for the first one

In [77]:
query = 'SELECT title, id, wp_publication_date from meta where "LCC" like "%PR%" and languages like "%en%" and (' + \
" ".join(queryFrags) + ")"

In [80]:
dataWp = cur.execute(query).fetchall()

In [88]:
wpAdditions = [x for x in dataWp if x[1] not in [book[1] for book in data]]
sorted(wpAdditions) 

[("Barnaby Rudge: A Tale of the Riots of 'Eighty", '917.0', ''),
 ('Cranford', '394.0', ''),
 ('Georgian Poetry 1920-22', '9640.0', ''),
 ('Hypatia — or New Foes with an Old Face', '6308.0', ''),
 ('Lavengro: The Scholar - The Gypsy - The Priest, Vol. 2 (of 2)',
  '22878.0',
  ''),
 ('Lavengro: The Scholar, The Gypsy, The Priest', '30792.0', ''),
 ('Lavengro: The Scholar, the Gypsy, the Priest', '23287.0', ''),
 ('Lavengro: The Scholar, the Gypsy, the Priest', '452.0', ''),
 ('Lavengro: the Scholar - the Gypsy - the Priest', '20198.0', ''),
 ('Loss and Gain: The Story of a Convert', '24574.0', ''),
 ('Mary Barton', '2153.0', ''),
 ('My Lady Ludlow', '2524.0', ''),
 ('North and South', '4276.0', ''),
 ('Percival Keene', '21572.0', ''),
 ('Phantastes: A Faerie Romance for Men and Women', '325.0', ''),
 ('Ruth', '4275.0', ''),
 ('The Caxtons: A Family Picture — Complete', '7605.0', ''),
 ("The Haunted Man and the Ghost's Bargain", '644.0', '1848-12-19'),
 ('The Heir of Redclyffe', '2505.0

Dedupe this list, too. 

In [100]:
wpDupes = [22878, 23287, 452, 20198, 1480]

In [101]:
allDupes = [str(float(x)) for x in (dupes + wpDupes)]

In [102]:
allDupes

['19337.0',
 '46.0',
 '30368.0',
 '2432.0',
 '6054.0',
 '20795.0',
 '678.0',
 '12954.0',
 '22496.0',
 '4401.0',
 '4402.0',
 '4404.0',
 '22878.0',
 '23287.0',
 '452.0',
 '20198.0',
 '1480.0']

Next, manually go through Wikipedia pages like [1840 British Novels](https://en.wikipedia.org/wiki/Category:1840_British_novels), and grab any books that have etexts on Project Gutenberg that we haven't already found automatically. 

In [89]:
manualAdditions = [
    37750, # Guy Fawkes, 1840
    16215, # Jack Sheppard, 1840
    21575, # Poor Jack, 1840
    11082, # Old Saint Paul's, 1841
    31004, # Ten thousand a-year, 1841
    36481, # Auriol, 1844
    20004, # Tancred, 1847
    15493, # Lancashire Witches, 1849
    7265, # Pendennis, 1850
    46733, # String of Pearls, 1850
    4644,  # Verdant Green, 1853
    38949, # Charles Auchester, 1853
    30664, # Callista, 1855
    56621, # Aurora Leigh, 1856
    646 # Coral Island, 1858   
]

In [103]:
masterListIDs = [x[1] for x in data] + [x[1] for x in wpAdditions] + [str(float(x)) for x in manualAdditions]
masterListIDs = [x for x in masterListIDs if x not in allDupes]

In [105]:
len(masterListIDs)

159

In [112]:
metadata = pd.read_sql_query('SELECT * from meta where id in (%s)' % ','.join('?'*len(masterListIDs)), conn, params=masterListIDs)

In [114]:
metadata.to_hdf('pr-1840-1860.hdf', 'meta')

/home/jon/.local/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['', 'LCC', 'author', 'authoryearofbirth', 'authoryearofdeath', 'downloads', 'formats', 'id', 'languages', 'lcsh', 'title', 'type', '_repo', '_version', 'alternative_title', 'contributor', 'covers', 'creator', 'description', 'edition_identifiers', 'edition_note', 'gutenberg_bookshelf', 'gutenberg_issued', 'gutenberg_type', 'identifiers', 'jmdate', 'subjects', 'language_note', 'production_note', 'publication_date', 'publication_note', 'publisher', 'rights', 'rights_url', 'series_note', 'summary', 'tableOfContents', 'titlepage_image', 'url', 'wikipedia', 'filename', 'releaseDate', 'wp_publication_date', 'wp_subjects', 'wp_info', 'wp_literary_genres', 'gr_rating', 'gr_numReviews', 'gr_pubDate', 'gr_id', 'gr_info', 'gr_author_id', 'gr_image_url'

In [115]:
texts = pd.read_sql_query('SELECT * from text where id in (%s)' % ','.join('?'*len(masterListIDs)), conn, params=masterListIDs)

In [116]:
len(texts)

147

In [118]:
texts.to_hdf('pr-1840-1860-texts.hdf', 'text')